In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [5]:
data = pd.read_excel('tesla_superchargers.xlsx', sheet_name='superchargers')

In [6]:
data.head()

,Supercharger,Street Address,City,State,Zip,Country,Stalls,kW,GPS,Elev(m),Open Date
0,"Buellton, CA",555 McMurray Rd,Buellton,CA,93427,USA,10,150.0,"34.61456, -120.188387",114,2013-07-13
1,"Corning, CA",950 Hwy 99,Corning,CA,96021,USA,6,150.0,"39.926454, -122.198393",87,2013-10-18
2,"Barstow, CA",2812 Lenwood Rd,Barstow,CA,92311,USA,16,150.0,"34.849129, -117.085446",725,2012-11-19
3,"Tifton, GA",1310 U.S. 82,Tifton,GA,31794,USA,8,150.0,"31.448847, -83.53221",92,2014-07-10
4,"Roseville, CA",1151 Galleria Blvd,Roseville,CA,95678,USA,7,150.0,"38.771208, -121.266149",66,2014-04-29


In [7]:
data

,Supercharger,Street Address,City,State,Zip,Country,Stalls,kW,GPS,Elev(m),Open Date
0,"Buellton, CA",555 McMurray Rd,Buellton,CA,93427,USA,10,150.0,"34.61456, -120.188387",114,2013-07-13
1,"Corning, CA",950 Hwy 99,Corning,CA,96021,USA,6,150.0,"39.926454, -122.198393",87,2013-10-18
2,"Barstow, CA",2812 Lenwood Rd,Barstow,CA,92311,USA,16,150.0,"34.849129, -117.085446",725,2012-11-19
3,"Tifton, GA",1310 U.S. 82,Tifton,GA,31794,USA,8,150.0,"31.448847, -83.53221",92,2014-07-10
4,"Roseville, CA",1151 Galleria Blvd,Roseville,CA,95678,USA,7,150.0,"38.771208, -121.266149",66,2014-04-29
...,...,...,...,...,...,...,...,...,...,...,...
5871,"Beijing - Haidian B+W Checker, China","69-1 Banjing Rd, Haidian District",Beijing,Beijing,0,China,2,120.0,"39.951088, 116.281837",58,1900-01-01
5872,"Cagnes-sur-Mer – Polygone Riviera P2, France",27-9 Rue de la Grange Rimade,Cagnes-sur-Mer,Provence-Alpes-Côte d'Azur,6800,France,4,125.0,"43.666884, 7.124728",30,1900-01-01
5873,"Naarden, Netherlands",IJsselmeerweg 3,Naarden,North Holland,1411,Netherlands,32,150.0,"52.30858, 5.141127",2,1900-01-01
5874,"Karlskrona, Sweden",Landbron 1 (private initiative but free for al...,Karlskrona,Blekinge,37133,Sweden,2,60.0,"56.165403, 15.585989",4,1900-01-01


In [10]:
us_data = data.loc[data['Country'] == 'USA']

In [11]:
us_data

,Supercharger,Street Address,City,State,Zip,Country,Stalls,kW,GPS,Elev(m),Open Date
0,"Buellton, CA",555 McMurray Rd,Buellton,CA,93427,USA,10,150.0,"34.61456, -120.188387",114,2013-07-13
1,"Corning, CA",950 Hwy 99,Corning,CA,96021,USA,6,150.0,"39.926454, -122.198393",87,2013-10-18
2,"Barstow, CA",2812 Lenwood Rd,Barstow,CA,92311,USA,16,150.0,"34.849129, -117.085446",725,2012-11-19
3,"Tifton, GA",1310 U.S. 82,Tifton,GA,31794,USA,8,150.0,"31.448847, -83.53221",92,2014-07-10
4,"Roseville, CA",1151 Galleria Blvd,Roseville,CA,95678,USA,7,150.0,"38.771208, -121.266149",66,2014-04-29
...,...,...,...,...,...,...,...,...,...,...,...
5856,"Fort Myers (closed), FL",9903 Gulf Coast Main St,Fort Myers,FL,33913,USA,8,150.0,"26.48564, -81.787136",6,1900-01-01
5857,"Rogers (SC), MN",22015 S Diamond Lake Rd,Rogers,MN,55374,USA,2,250.0,"45.199393, -93.560314",282,1900-01-01
5862,"Iowa, LA",800 V F Factory Outlet Dr,Iowa,LA,70647,USA,8,250.0,"30.247262, -93.010598",7,1900-01-01
5864,"Blue Ash (SC), OH",9111 Blue Ash Rd,Blue Ash,OH,45242,USA,8,150.0,"39.224429, -84.383415",258,1900-01-01


## Haversine function to calculate distance on the `Sphere`

In [12]:
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8 # Radius of Earth in miles

    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    return R * c

In [13]:
us_data.isnull().sum()

Supercharger      0
Street Address    0
City              0
State             0
Zip               0
Country           0
Stalls            0
kW                1
GPS               0
Elev(m)           0
Open Date         0
dtype: int64

## Calculating Distances between all SuperChargers

In [14]:
us_data[['Latitude', 'Longitude']] = us_data['GPS'].str.split(',', expand=True).astype(float)

distances = []

for i in range(len(us_data)):
    for j in range(i+1, len(us_data)):
        lat1, lon1 = us_data.iloc[i][['Latitude', 'Longitude']]
        lat2, lon2 = us_data.iloc[j][['Latitude', 'Longitude']]
        distance = haversine(lat1, lon1, lat2, lon2)
        distances.append({
            'Supercharger_1': us_data.iloc[i]['Supercharger'], 
            'Supercharger_2': us_data.iloc[j]['Supercharger'],
            'Distance_Miles': distance
        })

distance_df = pd.DataFrame(distances)   

/var/folders/3k/tnn35gr107sggvwln1123r5h0000gn/T/ipykernel_75736/1072940062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_data[['Latitude', 'Longitude']] = us_data['GPS'].str.split(',', expand=True).astype(float)
/var/folders/3k/tnn35gr107sggvwln1123r5h0000gn/T/ipykernel_75736/1072940062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_data[['Latitude', 'Longitude']] = us_data['GPS'].str.split(',', expand=True).astype(float)


In [15]:
distance_df

,Supercharger_1,Supercharger_2,Distance_Miles
0,"Buellton, CA","Corning, CA",383.267436
1,"Buellton, CA","Barstow, CA",176.932210
2,"Buellton, CA","Tifton, GA",2122.977502
3,"Buellton, CA","Roseville, CA",293.334245
4,"Buellton, CA","Oxnard, CA",63.201586
...,...,...,...
2561711,"Rogers (SC), MN","Blue Ash (SC), OH",624.564597
2561712,"Rogers (SC), MN","Burbank (SC), CA",1511.044381
2561713,"Iowa, LA","Blue Ash (SC), OH",789.522562
2561714,"Iowa, LA","Burbank (SC), CA",1499.135200


## Model S:

 - #### Long Range: 405 miles
 - #### Plaid: 396 miles


## Model X:

 - #### Long Range: 351 miles
 - #### Plaid: 333 miles

## Model 3:

 - #### Standard Range Plus: 263 miles
 - #### Long Range: 353 miles
 - #### Performance: 315 miles


## Model Y:

 - #### Long Range: 326 miles
 - #### Performance: 303 miles

## Deciding the next supercharger location

In [16]:
! pip install folium

  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/ae/6d/18a7546e1748ecdd6ed7cd00d3f183faf1df08bd4f5e5e0eb3e72458b862/folium-0.17.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/75/ca/6074ab4a04dd1a503201c18091b3426f3709670115fae316907a97f98d75/branca-0.7.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 260.2 kB/s eta 0:00:00a 0:00:01


In [18]:
locations = us_data[['Latitude', 'Longitude', 'Supercharger']]

m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

for _, location in locations.iterrows():
    folium.Marker(
        location = [location['Latitude'], location['Longitude']], 
        popup = location['Supercharger'],
    ).add_to(m)

In [19]:
m

## Highlighting the Gaps

In [31]:
# Extract and clean latitude and longitude
def clean_gps(gps):
    try:
        lat, lon = map(float, gps.split(','))
        return lat, lon
    except:
        return None, None

superchargers = us_data[['Supercharger', 'GPS']].dropna()
superchargers[['Latitude', 'Longitude']] = superchargers['GPS'].apply(lambda x: pd.Series(clean_gps(x)))

# Remove rows with invalid GPS data
superchargers = superchargers.dropna(subset=['Latitude', 'Longitude'])

# Create a dictionary for quick lookup of coordinates
supercharger_locations = superchargers.set_index('Supercharger')[['Latitude', 'Longitude']].to_dict('index')

In [34]:
supercharger_locations

{'Buellton, CA': {'Latitude': 34.61456, 'Longitude': -120.188387},
 'Corning, CA': {'Latitude': 39.926454, 'Longitude': -122.198393},
 'Barstow, CA': {'Latitude': 34.849129, 'Longitude': -117.085446},
 'Tifton, GA': {'Latitude': 31.448847, 'Longitude': -83.53221},
 'Roseville, CA': {'Latitude': 38.771208, 'Longitude': -121.266149},
 'Oxnard, CA': {'Latitude': 34.238654, 'Longitude': -119.177398},
 'Daytona Beach - Gateway North Dr, FL': {'Latitude': 29.223265,
  'Longitude': -81.09972},
 'Port St. Lucie, FL': {'Latitude': 27.313023, 'Longitude': -80.406688},
 'Dallas - Park Ln, TX': {'Latitude': 32.867639, 'Longitude': -96.767245},
 'El Centro, CA': {'Latitude': 32.760837, 'Longitude': -115.532486},
 'Rosemont, IL': {'Latitude': 41.975091, 'Longitude': -87.866501},
 'Indio, CA': {'Latitude': 33.741484, 'Longitude': -116.215019},
 'Savannah, GA': {'Latitude': 32.135507, 'Longitude': -81.212767},
 'Holbrook, AZ': {'Latitude': 34.922962, 'Longitude': -110.145558},
 'Mooresville, NC': {'La

In [45]:
model_ranges = {
    'Model S': 396,
    'Model X': 333,
    'Model 3': 263,
    'Model Y': 303
}


colors = {
    'Model S': 'blue',
    'Model X': 'green',
    'Model 3': 'red',
    'Model Y': 'orange'
}


gap_lines = {model: [] for model in model_ranges}

for model, range_ in model_ranges.items():
    filtered_df = distance_df[distance_df['Distance_Miles'] > range_]
    for supercharger_1, supercharger_2 in zip(filtered_df['Supercharger_1'], filtered_df['Supercharger_2']):
        loc1 = supercharger_locations.get(supercharger_1)
        loc2 = supercharger_locations.get(supercharger_2)

        if loc1 and loc2:
            loc1 = tuple(loc1.values())
            loc2 = tuple(loc2.values())
            gap_lines[model].append((loc1, loc2))

In [46]:
gap_lines

{'Model S': [((34.61456, -120.188387), (31.448847, -83.53221)),
  ((34.61456, -120.188387), (29.223265, -81.09972)),
  ((34.61456, -120.188387), (27.313023, -80.406688)),
  ((34.61456, -120.188387), (32.867639, -96.767245)),
  ((34.61456, -120.188387), (41.975091, -87.866501)),
  ((34.61456, -120.188387), (32.135507, -81.212767)),
  ((34.61456, -120.188387), (34.922962, -110.145558)),
  ((34.61456, -120.188387), (35.587472, -80.873)),
  ((34.61456, -120.188387), (33.443011, -112.556876)),
  ((34.61456, -120.188387), (32.943969, -112.733506)),
  ((34.61456, -120.188387), (41.041538, -73.671661)),
  ((34.61456, -120.188387), (41.245814, -73.008988)),
  ((34.61456, -120.188387), (29.108566, -81.034569)),
  ((34.61456, -120.188387), (43.63385, -95.595647)),
  ((34.61456, -120.188387), (36.07079, -79.511211)),
  ((34.61456, -120.188387), (44.520607, -103.890924)),
  ((34.61456, -120.188387), (33.485858, -80.475763)),
  ((34.61456, -120.188387), (41.7174, -86.1887)),
  ((34.61456, -120.18838

In [47]:
for model, lines in gap_lines.items():
    for loc1, loc2 in lines:
        folium.PolyLine(
            [loc1, loc2],
            color = colors[model],
            weight = 2.5,
            opacity = 1,
            tooltip = f"{model} gap"
        ).add_to(m)

In [ ]:
m.save('Supercharger_map.html')

In [22]:
distance_df

,Supercharger_1,Supercharger_2,Distance_Miles
0,"Buellton, CA","Corning, CA",383.267436
1,"Buellton, CA","Barstow, CA",176.932210
2,"Buellton, CA","Tifton, GA",2122.977502
3,"Buellton, CA","Roseville, CA",293.334245
4,"Buellton, CA","Oxnard, CA",63.201586
...,...,...,...
2561711,"Rogers (SC), MN","Blue Ash (SC), OH",624.564597
2561712,"Rogers (SC), MN","Burbank (SC), CA",1511.044381
2561713,"Iowa, LA","Blue Ash (SC), OH",789.522562
2561714,"Iowa, LA","Burbank (SC), CA",1499.135200


In [23]:
# Convert to numpy arrays for faster computation
latitudes = us_data['Latitude'].values
longitudes = us_data['Longitude'].values
supercharger_names = us_data['Supercharger'].values

In [24]:
latitudes

array([34.61456 , 39.926454, 34.849129, ..., 30.247262, 39.224429,
       34.174712])

In [25]:
longitudes

array([-120.188387, -122.198393, -117.085446, ...,  -93.010598,
        -84.383415, -118.300826])

In [26]:
supercharger_names

array(['Buellton, CA', 'Corning, CA', 'Barstow, CA', ..., 'Iowa, LA',
       'Blue Ash (SC), OH', 'Burbank (SC), CA'], dtype=object)